# Head First TensorFlow


####  Author: Yue-Wen FANG,
#### Contact: fyuewen@gmail.com
####  Revision history: created in late August 2017, at New York Uniersity, Shanghai
#### New York University, Shanghai
#### Kyto University
#### In the begining, I read the book by Jiaxuan Li, but soon I realized it is not good for beginers, and hence I switch to read this book by Caicloud.

## 1. 深度学习简介

## 2. TensorFlow 环境搭建

Please refere to my notes in the directory of JiaxuanLi-book2017

## 3. TensorFlow 入门

### 3.1 TensorFlow 计算模型

In [6]:
import tensorflow as tf
a = tf.constant([1.0,2.0], name = "a")
b = tf.constant([1.0,2.0], name = "b")
c = a + b
with tf.Session() as sess:
    sess.run(c)
print(c)#

Tensor("add_3:0", shape=(2,), dtype=float32)
